# Applying filters to the imagery

This step is to preserve edges and remove vegetation

In [1]:
import numpy as np
from skimage.filters import median, wiener, gaussian
from skimage.morphology import disk
from sklearn import preprocessing
import scipy.misc


In [2]:
# %load ../data_preprocessing/functions.py
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

# A function to plot a matrix
def plot_image(data, colorspace='binary'):
    fig = plt.figure(figsize=(10,10)) 
    ax = fig.add_subplot(111)
    ax.imshow(data, cmap=colorspace)
    
    
def normalize_array(array):
    array_min = array.min(axis=(0, 1), keepdims=True)
    array_max = array.max(axis=(0, 1), keepdims=True)
    normalized_array = (array - array_min)/(array_max - array_min)
    
    return normalized_array

def normalize_with_preset(array, max_value, min_value):
    normalized_array = (array - min_value)/(max_value - min_value)
    
    return normalized_array


def recreate_image(labels, w, h):
    """Recreate the (compressed) image from the code book & labels"""
    image = np.zeros((w, h))
    label_idx = 0
    for i in range(w):
        for j in range(h):
            image[i][j] = labels[label_idx]
            label_idx += 1
    return image

def overlay_images(image1, image2, colorspace1='binary', colorspace2='magma', transparency=0.5):
    plt.imshow(image1, cmap=colorspace1, interpolation='nearest')
    plt.imshow(image2, cmap=colorspace2, alpha=transparency, interpolation='bilinear')
    plt.show()
    
def flatten_normalize_neighbors(training_set, global_max=233, global_min=69):
    flattened_neighbors = []
    neighbors = training_set['neighbors']

    for i in range(len(neighbors)):
        numpy_array = np.array(neighbors[i]).reshape(-1, 1)
        normalized_neighbors = normalize_with_preset(numpy_array, global_max, global_min)
        flattened_neighbors.append(normalized_neighbors)
    
    return np.array(flattened_neighbors).reshape((len(flattened_neighbors), -1))

def extract_values_from_training_data_locations(training_data, image_channel):
    data_values = []
    
    for row in training_data:
        data_values.append(np.array([(image_channel[row[0], row[1]]), row[2]]))
        
    data_values = np.array(data_values)
    
    return data_values

def extract_neighborhood_values_from_training_data_locations(training_data, image_channel):
    data_values = []
    
    for row in training_data:
        data_values.append(np.array([(image_channel[row[0], row[1]]), row[2]]))
        
    data_values = np.array(data_values)
    
    return data_values

def flatten_neighbors(neighbors):
    data_values = []
    
    for neighbor in neighbors:
        #data_values.append(neighbor[0].reshape(-1, 1))
        data_values.append(neighbor[0].reshape(-1, 1))
    
    data_values = np.array(data_values)
    
    return data_values

def extract_labels_from_set(dataset):
    labels = []
    for neighbor in training_set:
        labels.append(neighbor[1])

    return np.array(labels).flatten()

## Load the colorspace converted data

In [3]:
original_enhanced_image = np.load('./data_preprocessing/numpy_data/indios_a.npy')
indios_b_hsv_image = np.load('./data_preprocessing/numpy_data/indios_a_hsv.npy')

In [7]:
r = original_enhanced_image[:, :, 0]
g = original_enhanced_image[:, :, 1]
b = original_enhanced_image[:, :, 2]

h = indios_b_hsv_image[:, :, 0]
s = indios_b_hsv_image[:, :, 1]
v = indios_b_hsv_image[:, :, 2]

In [8]:
# f, axarr = plt.subplots(1,3)
# axarr[0].imshow(r)
# axarr[1].imshow(g)
# axarr[2].imshow(b)

## Save images for document

In [5]:
scipy.misc.imsave('./data_preprocessing/numpy_data/indios_b/original_rgb.jpg', original_enhanced_image)
scipy.misc.imsave('./data_preprocessing/numpy_data/indios_b/original_hsv.jpg', indios_b_hsv_image)

In [ ]:
import gc

## Apply median filtering

In [9]:
filtered_r = median(r, disk(20))
filtered_g = median(g, disk(20))
filtered_b = median(b, disk(20))

In [10]:
filtered_h = median(h, disk(20))
filtered_s = median(s, disk(20))
filtered_v = median(v, disk(20))

D:\Users\gavargas\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [11]:
rgb_filtered = np.dstack([filtered_r, filtered_g, filtered_b])
hsv_filtered = np.dstack([filtered_h, filtered_s, filtered_v])

In [12]:
# rgb_subset = original_enhanced_image[0:4000, 0:4000, :]
# hsv_subset = indios_b_hsv_image[0:4000, 0:4000, :]

# rgb_filtered_subset = rgb_filtered[0:4000, 0:4000, :]
# hsv_filtered_subset = hsv_filtered[0:4000, 0:4000, :]

In [13]:
scipy.misc.imsave('./data_preprocessing/numpy_data/indios_a/original_filtered_rgb.jpg', rgb_filtered)
scipy.misc.imsave('./data_preprocessing/numpy_data/indios_a/original_filtered_hsv.jpg', hsv_filtered)

## Normalize Images

In [14]:
normalized_filtered_rgb = normalize_array(rgb_filtered)
normalized_filtered_hsv = normalize_array(hsv_filtered)

# hsv_image = hsv_image.astype(np.uint16)
# lab_image = lab_image.astype(np.uint16)

## Storing the filtered data

In [17]:
np.save('./data_preprocessing/numpy_data/indios_a/normalized_filtered_rgb.npy', normalized_filtered_rgb)
np.save('./data_preprocessing/numpy_data/indios_a/normalized_filtered_hsv.npy', normalized_filtered_hsv)